In [ ]:
from requests import get, post
from os import getenv

token=getenv("TOKEN")
headers = { "Authorization": f"Bearer {token}" }

In [ ]:
from requests import post
from os import getenv

base_url = getenv("BASE_URL")
model_name = 'distance'
response = get(url=f"https://distance-prediction-rest{base_url}/api/model_registry/v1alpha3/registered_models",
               headers=headers)
models = response.json()['items']
model_exists = [x for x in models if x['name'] == model_name]

if not model_exists:
    data = {
        "name": model_name
    }
    
    new_model = post(url=f"https://distance-prediction-rest{base_url}/api/model_registry/v1alpha3/registered_models",
               headers=headers, json=data)
    new_model
    print("Created new model")
    model_id = new_model.json()['id']
else:
    model_id = model_exists[0]['id']

In [ ]:
version = "1.0.1"
data = {
    "author": "kube:admin",
    "name": version,
    "registeredModelId": model_id,
    "state": "LIVE"
}

response = post(url=f"https://distance-prediction-rest{base_url}/api/model_registry/v1alpha3/model_versions",
               headers=headers,
               json=data)
model_version_id = response.json()['id']
response

In [ ]:
data = {
    'artifactType': 'model-artifact',
    "storageKey": "aws-connection-my-storage",
    "name": version,
    "modelFormatName": "onnx",
    "modelFormatVersion": "1",
    "uri": "s3://my-storage/models/distance/1?endpoint=https%3A%2F%2Fminio-s3-distance-prediction.apps.cluster-7px8r.7px8r.sandbox2276.opentlc.com&defaultRegion=us-east-1",
    "description": "Test",
    "state": "LIVE"
}

response = post(url=f"https://distance-prediction-rest{base_url}/api/model_registry/v1alpha3/model_versions/{model_version_id}/artifacts",
               headers=headers,
               json=data)
response